In [ ]:
import numpy as np
import pandas as pd
df = pd.read_csv('/content/f1_pitstops_2018_2024.csv')

In [ ]:
# -------------------------------------------------
# 3. Exact mapping of *all* broken patterns you may have
# -------------------------------------------------
name_fix = {
    # Kimi Räikkönen
    'Kimi RÃƒÂ¤ikkÃƒÂ¶nen' : 'Kimi Räikkönen',
    'Kimi RÃ¤ikkÃ¶nen': 'Kimi Räikkönen',
    'Kimi RÃƒÂ¤ikkÃƒÂ¶nen': 'Kimi Räikkönen',
    'Kimi RÃ¤ikkÃ¶nen': 'Kimi Räikkönen',
    'Kimi RÃ¤ikkÃ¶nen': 'Kimi Räikkönen',

    # Nico Hülkenberg
    'Nico HÃ¼lkenberg':   'Nico Hülkenberg',
    'Nico HÃƒÂ¼lkenberg': 'Nico Hülkenberg',
    'Nico HÃ¼lkenberg':   'Nico Hülkenberg',

    # Sergio Pérez
    'Sergio PÃ©rez':      'Sergio Pérez',
    'Sergio PÃƒÂ©rez':    'Sergio Pérez',
    'Sergio PÃ©rez':      'Sergio Pérez',

    # Add any others you discover
}

df_raw['Driver'] = df_raw['Driver'].replace(name_fix, regex=False)

print("\nAfter replacement – unique driver names (first 15):")
print(sorted(df_raw['Driver'].unique())[:15])

In [ ]:
print(df['Driver'].unique())

In [ ]:
name_corrections = {
    'Kimi RÃƒÂ¤ikkÃƒÂ¶nen' : 'Kimi Räikkönen',
    'Nico HÃƒÂ¼lkenberg' : 'Nico Hülkenberg',
    'Sergio PÃƒÂ©rez' : 'Sergio Pérez',
  }


In [ ]:
df['Driver'] = df['Driver'].replace(name_corrections)

In [ ]:
df

In [ ]:
df.info()

In [ ]:
df['Time_of_race'] = pd.to_datetime(df['Time_of_race'], errors='coerce')
df = df.drop(columns=['Abbreviation', 'Total Pit Stops'])
df['Date'] = pd.to_datetime(df['Date'])

In [ ]:
df['Stint'] = df['Stint'].fillna(0).astype(int)

In [ ]:
df['Pit_Lap'] = df['Pit_Lap'].replace('', pd.NA)
df['Pit_Lap'] = df['Pit_Lap'].fillna(df['Laps'] - df['Stint Length'])

In [ ]:
df

In [ ]:
df['Pit_Lap'].isna().sum()

In [ ]:
(df['Pit_Lap'] == '').sum()

In [ ]:
df['Pit_Lap'].isna().sum() + (df['Pit_Lap'] == '').sum()

In [ ]:
df[df['Pit_Lap'].isna() | (df['Pit_Lap'] == '')]

In [ ]:
# Replace NaN in Pit_Lap using (Laps - Stint Length)
df.loc[df['Pit_Lap'].isna(), 'Pit_Lap'] = df['Laps'] - df['Stint Length']

In [ ]:
print(df['Pit_Time'].isna().sum())

In [ ]:
(df['Circuit'] == '').sum()

In [ ]:
df[df['Race Name'].isna()]

In [ ]:
print(df['Circuit'].unique())

In [ ]:
name_corrections = {
    'AutÃƒÂ³dromo Hermanos RodrÃƒÂ­guez' : 'Autódromo Hermanos Rodríguez',
    'NÃƒÂ¼rburgring' : 'Nürburgring',
    'AutÃƒÂ³dromo Internacional do Algarve' : 'Autódromo Internacional do Algarve',
    'AutÃƒÂ³dromo JosÃƒÂ© Carlos Pace' : 'Interlagos'
}

In [ ]:
df['Circuit'] = df['Circuit'].replace(name_corrections)

In [ ]:
print(df['Tire Compound'].unique())

In [ ]:
df.drop(columns=['Race Name'], inplace=True)

In [ ]:
df

In [ ]:
cols = ['Date', 'Time_of_race', 'Country', 'Location',
        'AvgPitStopTime', 'Track_Temp_C', 'Humidity_%', 'Wind_Speed_KMH']

# Check if all columns have NaN in the same rows
same_rows = df[cols].isna().all(axis=1)  # rows where *all* are NaN
total_same = same_rows.sum()

print(f"✅ Number of rows where ALL these columns are NaN: {total_same}")
#print(f"Total NaN in each column: {df[cols].isna().sum().iloc[0]}")
print("➡️ Do they match exactly?", total_same == df[cols].isna().sum().iloc[0])

In [ ]:
cols = ['AvgPitStopTime', 'Lap Time Variation', 'Fast Lap Attempts', 'Driver Aggression Score']

# Check if all columns have NaN in the same rows
same_rows = df[cols].isna().all(axis=1)  # rows where *all* are NaN
total_same = same_rows.sum()

print(f"✅ Number of rows where ALL these columns are NaN: {total_same}")
print(f"Total NaN in each column: {df[cols].isna().sum().iloc[0]}")
print("➡️ Do they match exactly?", total_same == df[cols].isna().sum().iloc[0])

In [ ]:
# Define your two column groups
group1 = ['AvgPitStopTime', 'Lap Time Variation', 'Fast Lap Attempts', 'Driver Aggression Score']
group2 = ['Date', 'Time_of_race', 'Country', 'Location',
          'AvgPitStopTime', 'Track_Temp_C', 'Humidity_%', 'Wind_Speed_KMH']

# Step 1: Find the 185 rows where all 4 group1 columns are NaN
mask_group1 = df[group1].isna().all(axis=1)

# Step 2: Among those rows, check how many also have all group2 columns as NaN
mask_group2 = df[group2].isna().all(axis=1)

# Step 3: Compare overlap
overlap_count = (mask_group1 & mask_group2).sum()
total_group1 = mask_group1.sum()

print(f"✅ Rows where all 4 group1 columns are NaN: {total_group1}")
print(f"✅ Rows where those same rows also have all 8 group2 columns NaN: {overlap_count}")
print(f"➡️ Do they match exactly? {overlap_count == total_group1}")

# (Optional) Display overlapping rows
# df[mask_group1 & mask_group2]

In [ ]:
df[mask_group1 & mask_group2]

In [ ]:
# Drop rows where all 4 group1 columns AND all 8 group2 columns are NaN
df = df.drop(df[mask_group1 & mask_group2].index)

print("✅ Dropped rows where both group1 and group2 columns are completely NaN.")
print(f"Remaining rows in dataframe: {len(df)}")

In [ ]:
df.reset_index(drop=True, inplace=True)

In [ ]:
df

In [ ]:
# Show rows where Country or Location are missing
missing_rows = df[df['Country'].isna() | df['Location'].isna()][['Circuit', 'Country', 'Location']]
print(missing_rows)

In [ ]:
print(df['Circuit'].unique())

In [ ]:
print(df['Country'].unique())

In [ ]:
print(df['Location'].unique())

In [ ]:
# Example: fill based on Circuit
df.loc[df['Circuit'] == 'Albert Park Grand Prix Circuit', ['Country', 'Location']] = ['Australia', 'Melbourne']
df.loc[df['Circuit'] == 'Bahrain International Circuit', ['Country', 'Location']] = ['Bahrain', 'Sakhir']
df.loc[df['Circuit'] == 'Shanghai International Circuit', ['Country', 'Location']] = ['China', 'Shanghai']
df.loc[df['Circuit'] == 'Baku City Circuit', ['Country', 'Location']] = ['Azerbaijan', 'Baku']
df.loc[df['Circuit'] == 'Circuit de Barcelona-Catalunya', ['Country', 'Location']] = ['Spain', 'MontmelÃ³']
df.loc[df['Circuit'] == 'Circuit de Monaco', ['Country', 'Location']] = ['Monaco', 'Monte-Carlo']
df.loc[df['Circuit'] == 'Circuit Gilles Villeneuve', ['Country', 'Location']] = ['Canada', 'Montreal']
df.loc[df['Circuit'] == 'Circuit Paul Ricard', ['Country', 'Location']] = ['France', 'Le Castellet']
df.loc[df['Circuit'] == 'Red Bull Ring', ['Country', 'Location']] = ['Austria', 'Spielberg']
df.loc[df['Circuit'] == 'Silverstone Circuit', ['Country', 'Location']] = ['UK', 'Silverstone']
df.loc[df['Circuit'] == 'Hockenheimring', ['Country', 'Location']] = ['Germany', 'Hockenheim']
df.loc[df['Circuit'] == 'Hungaroring', ['Country', 'Location']] = ['Hungary', 'Budapest']
df.loc[df['Circuit'] == 'Circuit de Spa-Francorchamps', ['Country', 'Location']] = ['Belgium', 'Spa']
df.loc[df['Circuit'] == 'Autodromo Nazionale di Monza', ['Country', 'Location']] = ['Italy', 'Monza']
df.loc[df['Circuit'] == 'Marina Bay Street Circuit', ['Country', 'Location']] = ['Singapore', 'Marina Bay']
df.loc[df['Circuit'] == 'Sochi Autodrom', ['Country', 'Location']] = ['Russia', 'Sochi']
df.loc[df['Circuit'] == 'Suzuka Circuit', ['Country', 'Location']] = ['Japan', 'Suzuka']
df.loc[df['Circuit'] == 'Circuit of the Americas', ['Country', 'Location']] = ['USA', 'Austin']
df.loc[df['Circuit'] == 'Autódromo Hermanos Rodríguez', ['Country', 'Location']] = ['Mexico', 'Mexico City']
df.loc[df['Circuit'] == 'Yas Marina Circuit', ['Country', 'Location']] = ['UAE', 'Abu Dhabi']
df.loc[df['Circuit'] == 'Autodromo Internazionale del Mugello', ['Country', 'Location']] = ['Italy', 'Mugello']
df.loc[df['Circuit'] == 'Nürburgring', ['Country', 'Location']] = ['Germany', 'Nürburg']
df.loc[df['Circuit'] == 'Autódromo Internacional do Algarve', ['Country', 'Location']] = ['Portugal', 'Portimão']
df.loc[df['Circuit'] == 'Autodromo Enzo e Dino Ferrari', ['Country', 'Location']] = ['Italy', 'Imola']
df.loc[df['Circuit'] == 'Istanbul Park', ['Country', 'Location']] = ['Turkey', 'Istanbul']
df.loc[df['Circuit'] == 'Circuit Park Zandvoort', ['Country', 'Location']] = ['Netherlands', 'Zandvoort']
df.loc[df['Circuit'] == 'Losail International Circuit', ['Country', 'Location']] = ['Qatar', 'Al Daayen']
df.loc[df['Circuit'] == 'Jeddah Corniche Circuit', ['Country', 'Location']] = ['Saudi Arabia', 'Jeddah']
df.loc[df['Circuit'] == 'Miami International Autodrome', ['Country', 'Location']] = ['United States', 'Miami']
df.loc[df['Circuit'] == 'Las Vegas Strip Street Circuit', ['Country', 'Location']] = ['United States', 'Las Vegas']
df.loc[df['Circuit'] == 'Interlagos', ['Country', 'Location']] = ['Brazil', 'São Paulo']

In [ ]:
print(df['Location'].isna().sum())

In [ ]:
print(df['Season'].unique())

In [ ]:
df[df['Date'].isna()]

In [ ]:
missing_dates = df[df['Date'].isna()][['Season', 'Circuit']]
print(missing_dates.drop_duplicates().sort_values(['Season', 'Circuit']))

In [ ]:
# Group by year and list unique circuits
circuits_per_year = df.groupby('Season')['Circuit'].unique()

# Print in a readable format
for year, circuits in circuits_per_year.items():
    print(f"\n🏎️ {year} Circuits ({len(circuits)} total):")
    for c in circuits:
        print(f"  - {c}")

In [ ]:
# Step 1: Identify missing or NaT dates
missing_dates = df[df['Date'].isna()][['Season', 'Circuit']]
print("Total missing/NaT dates:", len(missing_dates))
print(missing_dates.drop_duplicates().sort_values(['Season', 'Circuit']))

# Step 2: Fill missing values manually using Year + Circuit
# Example: Replace below with actual race dates

#2018
df.loc[(df['Season'] == 2018) & (df['Circuit'] == 'Albert Park Grand Prix Circuit'), 'Date'] = '25-03-2018'
df.loc[(df['Season'] == 2018) & (df['Circuit'] == 'Bahrain International Circuit'), 'Date'] = '08-04-2018'
df.loc[(df['Season'] == 2018) & (df['Circuit'] == 'Shanghai International Circuit'), 'Date'] = '15-04-2018'
df.loc[(df['Season'] == 2018) & (df['Circuit'] == 'Baku City Circuit'), 'Date'] = '29-04-2018'
df.loc[(df['Season'] == 2018) & (df['Circuit'] == 'Circuit de Barcelona-Catalunya'), 'Date'] = '13-05-2018'
df.loc[(df['Season'] == 2018) & (df['Circuit'] == 'Circuit de Monaco'), 'Date'] = '27-05-2018'
df.loc[(df['Season'] == 2018) & (df['Circuit'] == 'Circuit Gilles Villeneuve'), 'Date'] = '10-06-2018'
df.loc[(df['Season'] == 2018) & (df['Circuit'] == 'Circuit Paul Ricard'), 'Date'] = '24-06-2018'
df.loc[(df['Season'] == 2018) & (df['Circuit'] == 'Red Bull Ring'), 'Date'] = '01-07-2018'
df.loc[(df['Season'] == 2018) & (df['Circuit'] == 'Silverstone Circuit'), 'Date'] = '08-07-2018'
df.loc[(df['Season'] == 2018) & (df['Circuit'] == 'Hockenheimring'), 'Date'] = '22-07-2018'
df.loc[(df['Season'] == 2018) & (df['Circuit'] == 'Hungaroring'), 'Date'] = '29-07-2018'
df.loc[(df['Season'] == 2018) & (df['Circuit'] == 'Circuit de Spa-Francorchamps'), 'Date'] = '26-08-2018'
df.loc[(df['Season'] == 2018) & (df['Circuit'] == 'Autodromo Nazionale di Monza'), 'Date'] = '02-09-2018'
df.loc[(df['Season'] == 2018) & (df['Circuit'] == 'Marina Bay Street Circuit'), 'Date'] = '16-09-2018'
df.loc[(df['Season'] == 2018) & (df['Circuit'] == 'Sochi Autodrom'), 'Date'] = '30-09-2018'
df.loc[(df['Season'] == 2018) & (df['Circuit'] == 'Suzuka Circuit'), 'Date'] = '07-10-2018'
df.loc[(df['Season'] == 2018) & (df['Circuit'] == 'Circuit of the Americas'), 'Date'] = '21-10-2018'
df.loc[(df['Season'] == 2018) & (df['Circuit'] == 'Autódromo Hermanos Rodríguez'), 'Date'] = '29-10-2018'
df.loc[(df['Season'] == 2018) & (df['Circuit'] == 'Interlagos'), 'Date'] = '11-11-2018'
df.loc[(df['Season'] == 2018) & (df['Circuit'] == 'Yas Marina Circuit'), 'Date'] = '25-11-2018'

#2019
df.loc[(df['Season'] == 2019) & (df['Circuit'] == 'Albert Park Grand Prix Circuit'), 'Date'] = '17-03-2019'
df.loc[(df['Season'] == 2019) & (df['Circuit'] == 'Bahrain International Circuit'), 'Date'] = '31-03-2019'
df.loc[(df['Season'] == 2019) & (df['Circuit'] == 'Shanghai International Circuit'), 'Date'] = '14-04-2019'
df.loc[(df['Season'] == 2019) & (df['Circuit'] == 'Baku City Circuit'), 'Date'] = '28-04-2019'
df.loc[(df['Season'] == 2019) & (df['Circuit'] == 'Circuit de Barcelona-Catalunya'), 'Date'] = '12-05-2019'
df.loc[(df['Season'] == 2019) & (df['Circuit'] == 'Circuit de Monaco'), 'Date'] = '26-05-2019'
df.loc[(df['Season'] == 2019) & (df['Circuit'] == 'Circuit Gilles Villeneuve'), 'Date'] = '09-06-2019'
df.loc[(df['Season'] == 2019) & (df['Circuit'] == 'Circuit Paul Ricard'), 'Date'] = '23-06-2019'
df.loc[(df['Season'] == 2019) & (df['Circuit'] == 'Red Bull Ring'), 'Date'] = '30-06-2019'
df.loc[(df['Season'] == 2019) & (df['Circuit'] == 'Silverstone Circuit'), 'Date'] = '14-07-2019'
df.loc[(df['Season'] == 2019) & (df['Circuit'] == 'Hockenheimring'), 'Date'] = '28-07-2019'
df.loc[(df['Season'] == 2019) & (df['Circuit'] == 'Hungaroring'), 'Date'] = '04-08-2019'
df.loc[(df['Season'] == 2019) & (df['Circuit'] == 'Circuit de Spa-Francorchamps'), 'Date'] = '01-09-2019'
df.loc[(df['Season'] == 2019) & (df['Circuit'] == 'Autodromo Nazionale di Monza'), 'Date'] = '08-09-2019'
df.loc[(df['Season'] == 2019) & (df['Circuit'] == 'Marina Bay Street Circuit'), 'Date'] = '22-09-2019'
df.loc[(df['Season'] == 2019) & (df['Circuit'] == 'Sochi Autodrom'), 'Date'] = '29-09-2019'
df.loc[(df['Season'] == 2019) & (df['Circuit'] == 'Suzuka Circuit'), 'Date'] = '13-10-2019'
df.loc[(df['Season'] == 2019) & (df['Circuit'] == 'Autódromo Hermanos Rodríguez'), 'Date'] = '28-10-2019'
df.loc[(df['Season'] == 2019) & (df['Circuit'] == 'Circuit of the Americas'), 'Date'] = '04-11-2019'
df.loc[(df['Season'] == 2018) & (df['Circuit'] == 'Interlagos'), 'Date'] = '17-11-2019'
df.loc[(df['Season'] == 2019) & (df['Circuit'] == 'Yas Marina Circuit'), 'Date'] = '01-12-2019'

#2020
#df.loc[(df['Season'] == 2020) & (df['Circuit'] == 'Red Bull Ring'), 'Date'] = '05-07-2020'
df.loc[(df['Season'] == 2020) & (df['Circuit'] == 'Hungaroring'), 'Date'] = '19-07-2020'
df.loc[(df['Season'] == 2020) & (df['Circuit'] == 'Silverstone Circuit'), 'Date'] = '02-08-2020'
df.loc[(df['Season'] == 2020) & (df['Circuit'] == 'Circuit de Barcelona-Catalunya'), 'Date'] = '16-08-2020'
df.loc[(df['Season'] == 2020) & (df['Circuit'] == 'Circuit de Spa-Francorchamps'), 'Date'] = '30-08-2020'
df.loc[(df['Season'] == 2020) & (df['Circuit'] == 'Autodromo Nazionale di Monza'), 'Date'] = '06-09-2020'
df.loc[(df['Season'] == 2020) & (df['Circuit'] == 'Autodromo Internazionale del Mugello'), 'Date'] = '13-09-2020'
df.loc[(df['Season'] == 2020) & (df['Circuit'] == 'Sochi Autodrom'), 'Date'] = '27-09-2020'
df.loc[(df['Season'] == 2020) & (df['Circuit'] == 'Nürburgring'), 'Date'] = '11-10-2020'
df.loc[(df['Season'] == 2020) & (df['Circuit'] == 'Autódromo Internacional do Algarve'), 'Date'] = '25-10-2020'
df.loc[(df['Season'] == 2020) & (df['Circuit'] == 'Autodromo Enzo e Dino Ferrari'), 'Date'] = '01-11-2020'
df.loc[(df['Season'] == 2020) & (df['Circuit'] == 'Istanbul Park'), 'Date'] = '15-11-2020'
df.loc[(df['Season'] == 2020) & (df['Circuit'] == 'Bahrain International Circuit'), 'Date'] = '29-11-2020'
df.loc[(df['Season'] == 2020) & (df['Circuit'] == 'Yas Marina Circuit'), 'Date'] = '13-12-2020'

#2021
df.loc[(df['Season'] == 2021) & (df['Circuit'] == 'Bahrain International Circuit'), 'Date'] = '28-03-2021'
df.loc[(df['Season'] == 2021) & (df['Circuit'] == 'Autodromo Enzo e Dino Ferrari'), 'Date'] = '18-04-2021'
df.loc[(df['Season'] == 2021) & (df['Circuit'] == 'Autódromo Internacional do Algarve'), 'Date'] = '02-05-2021'
df.loc[(df['Season'] == 2021) & (df['Circuit'] == 'Circuit de Barcelona-Catalunya'), 'Date'] = '09-05-2021'
df.loc[(df['Season'] == 2021) & (df['Circuit'] == 'Circuit de Monaco'), 'Date'] = '23-05-2021'
df.loc[(df['Season'] == 2021) & (df['Circuit'] == 'Baku City Circuit'), 'Date'] = '06-06-2021'
df.loc[(df['Season'] == 2021) & (df['Circuit'] == 'Circuit Paul Ricard'), 'Date'] = '20-06-2021'
#df.loc[(df['Season'] == 2021) & (df['Circuit'] == 'Red Bull Ring'), 'Date'] = ''
df.loc[(df['Season'] == 2021) & (df['Circuit'] == 'Silverstone Circuit'), 'Date'] = '18-07-2021'
df.loc[(df['Season'] == 2021) & (df['Circuit'] == 'Hungaroring'), 'Date'] = '01-08-2021'
df.loc[(df['Season'] == 2021) & (df['Circuit'] == 'Circuit de Spa-Francorchamps'), 'Date'] = '29-08-2021'
df.loc[(df['Season'] == 2021) & (df['Circuit'] == 'Circuit Park Zandvoort'), 'Date'] = '05-09-2021'
df.loc[(df['Season'] == 2021) & (df['Circuit'] == 'Autodromo Nazionale di Monza'), 'Date'] = '12-09-2021'
df.loc[(df['Season'] == 2021) & (df['Circuit'] == 'Sochi Autodrom'), 'Date'] = '26-09-2021'
df.loc[(df['Season'] == 2021) & (df['Circuit'] == 'Istanbul Park'), 'Date'] = '10-10-2021'
df.loc[(df['Season'] == 2021) & (df['Circuit'] == 'Circuit of the Americas'), 'Date'] = '25-10-2021'
df.loc[(df['Season'] == 2021) & (df['Circuit'] == 'Losail International Circuit'), 'Date'] = '21-10-2021'
df.loc[(df['Season'] == 2021) & (df['Circuit'] == 'Jeddah Corniche Circuit'), 'Date'] = '05-12-2021'
df.loc[(df['Season'] == 2021) & (df['Circuit'] == 'Yas Marina Circuit'), 'Date'] = '12-12-2021'

#2022
df.loc[(df['Season'] == 2022) & (df['Circuit'] == 'Bahrain International Circuit'), 'Date'] = '20-03-2022'
df.loc[(df['Season'] == 2022) & (df['Circuit'] == 'Jeddah Corniche Circuit'), 'Date'] = '27-03-2022'
df.loc[(df['Season'] == 2022) & (df['Circuit'] == 'Albert Park Grand Prix Circuit'), 'Date'] = '10-04-2022'
df.loc[(df['Season'] == 2022) & (df['Circuit'] == 'Autodromo Enzo e Dino Ferrari'), 'Date'] = '24-04-2022'
df.loc[(df['Season'] == 2022) & (df['Circuit'] == 'Miami International Autodrome'), 'Date'] = '09-05-2022'
df.loc[(df['Season'] == 2022) & (df['Circuit'] == 'Circuit de Barcelona-Catalunya'), 'Date'] = '22-05-2022'
df.loc[(df['Season'] == 2022) & (df['Circuit'] == 'Circuit de Monaco'), 'Date'] = '29-05-2022'
df.loc[(df['Season'] == 2022) & (df['Circuit'] == 'Baku City Circuit'), 'Date'] = '12-06-2022'
df.loc[(df['Season'] == 2022) & (df['Circuit'] == 'Circuit Gilles Villeneuve'), 'Date'] = '19-06-2022'
df.loc[(df['Season'] == 2022) & (df['Circuit'] == 'Silverstone Circuit'), 'Date'] = '03-07-2022'
df.loc[(df['Season'] == 2022) & (df['Circuit'] == 'Red Bull Ring'), 'Date'] = '10-07-2022'
df.loc[(df['Season'] == 2022) & (df['Circuit'] == 'Circuit Paul Ricard'), 'Date'] = '24-07-2022'
df.loc[(df['Season'] == 2022) & (df['Circuit'] == 'Hungaroring'), 'Date'] = '31-07-2022'
df.loc[(df['Season'] == 2022) & (df['Circuit'] == 'Circuit de Spa-Francorchamps'), 'Date'] = '28-08-2022'
df.loc[(df['Season'] == 2022) & (df['Circuit'] == 'Circuit Park Zandvoort'), 'Date'] = '04-09-2022'
df.loc[(df['Season'] == 2022) & (df['Circuit'] == 'Autodromo Nazionale di Monza'), 'Date'] = '11-09-2022'
df.loc[(df['Season'] == 2022) & (df['Circuit'] == 'Marina Bay Street Circuit'), 'Date'] = '02-10-2022'
df.loc[(df['Season'] == 2022) & (df['Circuit'] == 'Suzuka Circuit'), 'Date'] = '09-10-2022'
df.loc[(df['Season'] == 2022) & (df['Circuit'] == 'Circuit of the Americas'), 'Date'] = '24-10-2022'
df.loc[(df['Season'] == 2022) & (df['Circuit'] == 'Autódromo Hermanos Rodríguez'), 'Date'] = '31-10-2022'
df.loc[(df['Season'] == 2022) & (df['Circuit'] == 'Interlagos'), 'Date'] = '13-11-2022'
df.loc[(df['Season'] == 2022) & (df['Circuit'] == 'Yas Marina Circuit'), 'Date'] = '20-11-2022'

#2023
df.loc[(df['Season'] == 2023) & (df['Circuit'] == 'Bahrain International Circuit'), 'Date'] = '05-03-2023'
df.loc[(df['Season'] == 2023) & (df['Circuit'] == 'Jeddah Corniche Circuit'), 'Date'] = '19-03-2023'
df.loc[(df['Season'] == 2023) & (df['Circuit'] == 'Albert Park Grand Prix Circuit'), 'Date'] = '02-04-2023'
df.loc[(df['Season'] == 2023) & (df['Circuit'] == 'Baku City Circuit'), 'Date'] = '30-04-2023'
df.loc[(df['Season'] == 2023) & (df['Circuit'] == 'Miami International Autodrome'), 'Date'] = '08-05-2023'
df.loc[(df['Season'] == 2023) & (df['Circuit'] == 'Circuit de Monaco'), 'Date'] = '28-05-2023'
df.loc[(df['Season'] == 2023) & (df['Circuit'] == 'Circuit de Barcelona-Catalunya'), 'Date'] = '04-06-2023'
df.loc[(df['Season'] == 2023) & (df['Circuit'] == 'Circuit Gilles Villeneuve'), 'Date'] = '18-06-2023'
df.loc[(df['Season'] == 2023) & (df['Circuit'] == 'Red Bull Ring'), 'Date'] = '02-07-2023'
df.loc[(df['Season'] == 2023) & (df['Circuit'] == 'Silverstone Circuit'), 'Date'] = '09-07-2023'
df.loc[(df['Season'] == 2023) & (df['Circuit'] == 'Hungaroring'), 'Date'] = '23-07-2023'
df.loc[(df['Season'] == 2023) & (df['Circuit'] == 'Circuit de Spa-Francorchamps'), 'Date'] = '30-07-2023'
df.loc[(df['Season'] == 2023) & (df['Circuit'] == 'Circuit Park Zandvoort'), 'Date'] = '27-08-2023'
df.loc[(df['Season'] == 2023) & (df['Circuit'] == 'Autodromo Nazionale di Monza'), 'Date'] = '03-09-2023'
df.loc[(df['Season'] == 2023) & (df['Circuit'] == 'Marina Bay Street Circuit'), 'Date'] = '17-09-2023'
df.loc[(df['Season'] == 2023) & (df['Circuit'] == 'Suzuka Circuit'), 'Date'] = '24-09-2023'
df.loc[(df['Season'] == 2023) & (df['Circuit'] == 'Losail International Circuit'), 'Date'] = '08-10-2023'
df.loc[(df['Season'] == 2023) & (df['Circuit'] == 'Circuit of the Americas'), 'Date'] = '23-10-2023'
df.loc[(df['Season'] == 2023) & (df['Circuit'] == 'Autódromo Hermanos Rodríguez'), 'Date'] = '30-10-2023'
df.loc[(df['Season'] == 2023) & (df['Circuit'] == 'Interlagos'), 'Date'] = '05-11-2023'
df.loc[(df['Season'] == 2023) & (df['Circuit'] == 'Las Vegas Strip Street Circuit'), 'Date'] = '19-11-2023'
df.loc[(df['Season'] == 2023) & (df['Circuit'] == 'Yas Marina Circuit'), 'Date'] = '26-11-2023'

#2024
df.loc[(df['Season'] == 2024) & (df['Circuit'] == 'Bahrain International Circuit'), 'Date'] = '02-03-2024'
df.loc[(df['Season'] == 2024) & (df['Circuit'] == 'Jeddah Corniche Circuit'), 'Date'] = '09-03-2024'
df.loc[(df['Season'] == 2024) & (df['Circuit'] == 'Albert Park Grand Prix Circuit'), 'Date'] = '24-03-2024'
df.loc[(df['Season'] == 2024) & (df['Circuit'] == 'Suzuka Circuit'), 'Date'] = '07-04-2024'
df.loc[(df['Season'] == 2024) & (df['Circuit'] == 'Shanghai International Circuit'), 'Date'] = '21-04-2024'
df.loc[(df['Season'] == 2024) & (df['Circuit'] == 'Miami International Autodrome'), 'Date'] = '06-05-2024'
df.loc[(df['Season'] == 2024) & (df['Circuit'] == 'Autodromo Enzo e Dino Ferrari'), 'Date'] = '19-05-2024'
df.loc[(df['Season'] == 2024) & (df['Circuit'] == 'Circuit de Monaco'), 'Date'] = '26-05-2024'
df.loc[(df['Season'] == 2024) & (df['Circuit'] == 'Circuit Gilles Villeneuve'), 'Date'] = '09-06-2024'
df.loc[(df['Season'] == 2024) & (df['Circuit'] == 'Circuit de Barcelona-Catalunya'), 'Date'] = '23-06-2024'
df.loc[(df['Season'] == 2024) & (df['Circuit'] == 'Red Bull Ring'), 'Date'] = '30-06-2024'
df.loc[(df['Season'] == 2024) & (df['Circuit'] == 'Silverstone Circuit'), 'Date'] = '07-07-2024'
df.loc[(df['Season'] == 2024) & (df['Circuit'] == 'Hungaroring'), 'Date'] = '21-07-2024'
df.loc[(df['Season'] == 2024) & (df['Circuit'] == 'Circuit de Spa-Francorchamps'), 'Date'] = '28-07-2024'
df.loc[(df['Season'] == 2024) & (df['Circuit'] == 'Circuit Park Zandvoort'), 'Date'] = '25-08-2024'
df.loc[(df['Season'] == 2024) & (df['Circuit'] == 'Autodromo Nazionale di Monza'), 'Date'] = '01-09-2024'
df.loc[(df['Season'] == 2024) & (df['Circuit'] == 'Baku City Circuit'), 'Date'] = '15-09-2024'
df.loc[(df['Season'] == 2024) & (df['Circuit'] == 'Marina Bay Street Circuit'), 'Date'] = '22-09-2024'
df.loc[(df['Season'] == 2024) & (df['Circuit'] == 'Circuit of the Americas'), 'Date'] = '21-10-2024'
df.loc[(df['Season'] == 2024) & (df['Circuit'] == 'Autódromo Hermanos Rodríguez'), 'Date'] = '28-10-2024'
df.loc[(df['Season'] == 2024) & (df['Circuit'] == 'Interlagos'), 'Date'] = '03-11-2024'
df.loc[(df['Season'] == 2024) & (df['Circuit'] == 'Las Vegas Strip Street Circuit'), 'Date'] = '24-11-2024'
df.loc[(df['Season'] == 2024) & (df['Circuit'] == 'Losail International Circuit'), 'Date'] = '01-12-2024'
df.loc[(df['Season'] == 2024) & (df['Circuit'] == 'Yas Marina Circuit'), 'Date'] = '08-12-2024'



# Step 3: Ensure all values are proper datetime objects
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')

# Step 4: Check for any remaining NaT values
remaining_nat = df[df['Date'].isna()]
print("Remaining rows with NaT:", len(remaining_nat))
if not remaining_nat.empty:
    print(remaining_nat[['Year', 'Circuit']].drop_duplicates().sort_values(['Year', 'Circuit']))

# (Optional) Step 5: Save cleaned data
# df.to_csv("f1_cleaned.csv", index=False)

In [ ]:
# Find cells with exactly empty string ("")
empty_cells = (df == ' ').sum().sum()

print(f"Total empty-string cells in DataFrame: {empty_cells}")

In [ ]:
df[df['Pit_Time'].isna() | (df['Pit_Time'].astype(str).str.strip() == '')]

In [ ]:
cols = [
    'AvgPitStopTime', 'Time_of_race', 'Air_Temp_C', 'Track_Temp_C',
    'Humidity_%', 'Wind_Speed_KMH', 'Lap Time Variation', 'Tire Usage Aggression',
    'Fast Lap Attempts', 'Driver Aggression Score', 'Tire Compound',
    'Stint Length', 'Pit_Lap', 'Pit_Time'
]

df[df[cols].isna().all(axis=1) | (df[cols].astype(str).apply(lambda x: x.str.strip() == '').all(axis=1))]

In [ ]:
df['Location'] = df['Location'].replace('MontmelÃ³', 'Montmeló')

In [ ]:
# Save as CSV
df.to_csv('cleaned_pitstops.csv', index=False)

# For Colab users — this lets you download directly
from google.colab import files
files.download('cleaned_pitstops.csv')

In [ ]:
print(df['Time_of_race'])

In [ ]:
# Keep only the time portion from the datetime
df['Time_of_race'] = pd.to_datetime(df['Time_of_race'], errors='coerce').dt.time

In [ ]:
print(df['Time_of_race'])

In [ ]:
df.to_csv('cleaned_pitstops.csv', index=False)

# For Colab users — this lets you download directly
from google.colab import files
files.download('cleaned_pitstops.csv')

In [ ]:
# Count NaN or missing values
missing_count = df['Time_of_race'].isna().sum()
print("Number of missing or NaN values in Time_of_race:", missing_count)

In [ ]:
# Create a lookup dictionary from rows where Time_of_race is not missing
lookup = (
    df.dropna(subset=['Time_of_race'])
      .set_index(['Season', 'Round', 'Circuit'])['Time_of_race']
      .to_dict()
)

# Fill missing Time_of_race using the lookup
def fill_time(row):
    if pd.isna(row['Time_of_race']) or row['Time_of_race'] == '':
        key = (row['Season'], row['Round'], row['Circuit'])
        return lookup.get(key, row['Time_of_race'])
    return row['Time_of_race']

df['Time_of_race'] = df.apply(fill_time, axis=1)

In [ ]:
print("Remaining missing times:", df['Time_of_race'].isna().sum())

In [ ]:
print(df['Time_of_race'])

In [ ]:
# Find rows where Time_of_race is missing or empty
missing_time_rows = df[df['Time_of_race'].isna() | (df['Time_of_race'] == '')]

# Get unique combinations of Season, Round, and Circuit
unique_missing_races = missing_time_rows[['Season', 'Round', 'Circuit']].drop_duplicates()

# Display them
print("🏁 Unique races with missing Time_of_race:", len(unique_missing_races))
print(unique_missing_races)

In [ ]:

# Dictionary mapping (season, round, circuit) → time
time_updates = {
    (2018, 19, 'Autódromo Hermanos Rodríguez'): '19:10:00',
    (2019, 18, 'Autódromo Hermanos Rodríguez'): '19:10:00',
    (2020, 11, 'Nürburgring'): '12:10:00',
    (2020, 12, 'Autódromo Internacional do Algarve'): '13:10:00',
    (2021, 3, 'Autódromo Internacional do Algarve'): '14:00:00',
    (2022, 20, 'Autódromo Hermanos Rodríguez'): '20:00:00',
    (2023, 19, 'Autódromo Hermanos Rodríguez'): '20:00:00',
    (2024, 20, 'Autódromo Hermanos Rodríguez'): '20:00:00',
    (2024, 21, 'Interlagos'): '02:30:00'
}

# Fill missing times based on these combinations
for (season, rnd, circuit), race_time in time_updates.items():
    mask = (
        (df['Season'] == season) &
        (df['Round'] == rnd) &
        (df['Circuit'] == circuit)
    )
    df.loc[mask, 'Time_of_race'] = race_time

# (Optional) Verify updated rows
print(df.loc[df['Time_of_race'] == '05:10:00', ['Season', 'Round', 'Circuit', 'Time_of_race']])

In [ ]:
# Find rows where Time_of_race is missing or empty
missing_time_rows = df[df['Time_of_race'].isna() | (df['Time_of_race'] == '')]

# Get unique combinations of Season, Round, and Circuit
unique_missing_races = missing_time_rows[['Season', 'Round', 'Circuit']].drop_duplicates()

# Display them
print("🏁 Unique races with missing Time_of_race:", len(unique_missing_races))
print(unique_missing_races)

In [ ]:
print("Remaining missing times:", df['Time_of_race'].isna().sum())

In [ ]:
# Find rows where Time_of_race is missing or empty
missing_time_rows = df[df['Driver Aggression Score'].isna() | (df['Driver Aggression Score'] == '')]

# Get unique combinations of Season, Round, and Circuit
unique_missing_races = missing_time_rows[['Season', 'Round', 'Circuit']].drop_duplicates()

# Display them
print("🏁 Unique races with missing Time_of_race:", len(unique_missing_races))
print(unique_missing_races)

In [ ]:
# -------------------------------------------------
# 2. Define the four columns
# -------------------------------------------------
weather_cols = ['AvgPitStopTime', 'Lap Time Variation', 'Fast Lap Attempts', 'Driver Aggression Score']
# -------------------------------------------------
# 3. Count missing / empty / NaN per column
# -------------------------------------------------
missing_report = {}

for col in weather_cols:
    # 1. NaN (pandas null)
    nan_cnt = df[col].isna().sum()

    # 2. Empty strings (after converting to string)
    empty_cnt = (df[col].astype(str).str.strip() == '').sum()

    # 3. Total problematic rows
    total_bad = nan_cnt + empty_cnt

    missing_report[col] = {
        'NaN'        : nan_cnt,
        'Empty_str'  : empty_cnt,
        'Total_bad'  : total_bad,
        'Pct_bad'    : total_bad / len(df) * 100
    }

# -------------------------------------------------
# 4. Pretty-print the report
# -------------------------------------------------
import json, math
print("\n=== Missing-value report for weather columns ===\n")
for col, stats in missing_report.items():
    print(f"{col:15} | NaN: {stats['NaN']:>6} | Empty: {stats['Empty_str']:>6} | "
          f"Total bad: {stats['Total_bad']:>6} | "
          f"({stats['Pct_bad']:.2f}%)")

In [ ]:
# -------------------------------------------------
# 5. Bonus: Rows that have *any* missing weather value
# -------------------------------------------------
rows_with_any_missing = df[weather_cols].isna().any(axis=1).sum()
rows_all_missing      = df[weather_cols].isna().all(axis=1).sum()

print(f"\nRows with at least one missing weather value : {rows_with_any_missing:,}")
print(f"Rows with *all four* weather values missing  : {rows_all_missing:,}")

In [ ]:
print(f"Total rows in the dataset: {len(df):,}")

In [ ]:

cols_to_check = [
    'AvgPitStopTime',
    'Lap Time Variation',
    'Fast Lap Attempts',
    'Driver Aggression Score'
]
# 3. Count rows where *ALL FOUR* are NaN
all_missing_mask = df[cols_to_check].isna().all(axis=1)
rows_to_drop = all_missing_mask.sum()

print(f"Rows where ALL FOUR columns are missing: {rows_to_drop:,}")

# 4. Drop ONLY those rows
df_clean = df[~all_missing_mask].copy()

print(f"\nBefore: {len(df):,} rows")
print(f" After: {len(df_clean):,} rows  (removed {rows_to_drop:,} rows)")

# 5. Verify: no row has all four missing anymore
still_all_missing = df_clean[cols_to_check].isna().all(axis=1).sum()
print(f"\nRows with all four missing after drop: {still_all_missing}  (should be 0)")

# Optional: check how many rows still have *any* missing in these columns
any_missing_after = df_clean[cols_to_check].isna().any(axis=1).sum()
print(f"Rows with at least one missing (kept): {any_missing_after:,}")

In [ ]:
df = df_clean

In [ ]:
print(f"Total rows in the dataset: {len(df):,}")

In [ ]:
print("Remaining missing times:", df['Driver Aggression Score'].isna().sum())

In [ ]:
print("Remaining missing times:", df['AvgPitStopTime'].isna().sum())

In [ ]:
print("Remaining missing times:", df['Lap Time Variation'].isna().sum())

In [ ]:
print("Remaining missing times:", df['Fast Lap Attempts'].isna().sum())

In [ ]:
print("Remaining missing times:", df['Pit_Time'].isna().sum())

In [ ]:
print("Remaining missing times:", df['Stint Length'].isna().sum())

In [ ]:
print("Remaining missing times:", df['Tire Compound'].isna().sum())

In [ ]:
print("Remaining missing times:", df['Tire Usage Aggression'].isna().sum())

In [ ]:
print(f"Total rows in the dataset: {len(df):,}")

In [ ]:
# -------------------------------------------------
# 1. Define the 4 columns you care about
# -------------------------------------------------
cols = [
    'Tire Compound',
    'Stint Length',
    'Pit_Time',
    'Pit_Lap'
]

# -------------------------------------------------
# 2. Count missing values per column
# -------------------------------------------------
missing_counts = df[cols].isna().sum()
print("Missing value counts per column:")
print(missing_counts)

# -------------------------------------------------
# 3. Check if ALL counts are equal
# -------------------------------------------------
all_same_count = missing_counts.nunique() == 1
print(f"\nAll 4 columns have the SAME number of missing values? → {all_same_count}")

if all_same_count:
    n_missing = missing_counts.iloc[0]
    print(f"   → Each column has {n_missing:,} missing values")
else:
    print("   → Counts differ — see above")

# -------------------------------------------------
# 4. Check if missing values are in EXACTLY the same rows
# -------------------------------------------------
# Create a mask: True where ANY of the 4 is missing
any_missing_mask = df[cols].isna().any(axis=1)

# For each column, count how many of its missing rows are in the "any missing" set
same_rows = all(
    df[col].isna().sum() == (df[col].isna() & any_missing_mask).sum()
    for col in cols
)

print(f"\nMissing values occur in the EXACT SAME ROWS? → {same_rows}")

if same_rows:
    print(f"   → All {n_missing:,} missing entries are in the same {n_missing:,} rows")
else:
    print("   → Some columns have missing values in different rows")
    # Show which rows differ (optional)
    for col in cols:
        diff = df[col].isna() & ~any_missing_mask
        if diff.any():
            print(f"     • {col} has extra missing rows: {diff.sum()}")

# -------------------------------------------------
# 5. Bonus: Show the actual rows (first 5) with all 4 missing
# -------------------------------------------------
all_missing_rows = df[cols].isna().all(axis=1)
if all_missing_rows.any():
    print(f"\nSample rows where ALL 4 columns are missing (first 5):")
    print(df[all_missing_rows][['Season', 'Round', 'Driver'] + cols].head())
else:
    print("\nNo rows where all 4 are missing.")

In [ ]:
# -------------------------------------------------
# 1. Define the 4 columns
# -------------------------------------------------
cols = ['Tire Compound', 'Stint Length', 'Pit_Time', 'Pit_Lap']

# -------------------------------------------------
# 2. Verify they have the same number of missing values
# -------------------------------------------------
missing_counts = df[cols].isna().sum()
print("Missing counts per column:")
print(missing_counts)

all_same_count = missing_counts.nunique() == 1
print(f"\nAll 4 columns have same missing count? → {all_same_count}")

# -------------------------------------------------
# 3. Verify missing values are in the SAME rows
# -------------------------------------------------
any_missing = df[cols].isna().any(axis=1)
same_rows = all(
    (df[col].isna() & any_missing).sum() == df[col].isna().sum()
    for col in cols
)
print(f"Missing values in the SAME rows? → {same_rows}")

# -------------------------------------------------
# 4. Drop rows where ALL 4 are missing
# -------------------------------------------------
all_missing_mask = df[cols].isna().all(axis=1)
rows_to_drop = all_missing_mask.sum()
print(f"\nRows where ALL 4 columns are missing: {rows_to_drop:,}")

df_clean = df[~all_missing_mask].copy()
print(f"Before drop: {len(df):,} rows")
print(f" After drop: {len(df_clean):,} rows  (removed {rows_to_drop:,} rows)")

# -------------------------------------------------
# 5. Final verification
# -------------------------------------------------
still_all_missing = df_clean[cols].isna().all(axis=1).sum()
print(f"\nRows with all 4 missing after drop: {still_all_missing}  (should be 0)")

# Optional: Show how many rows still have partial missing (1–3 columns)
partial_missing = df_clean[cols].isna().any(axis=1).sum() - still_all_missing
print(f"Rows with 1–3 missing (kept): {partial_missing:,}")

In [ ]:
df = df[~df[['Tire Compound', 'Stint Length', 'Pit_Time', 'Pit_Lap']].isna().all(axis=1)].copy()
print(f"Final rows: {len(df):,}")

In [ ]:
print(f"Total rows in the dataset: {len(df):,}")

In [ ]:
print("Remaining missing times:", df['Pit_Lap'].isna().sum())

In [ ]:
print("Remaining missing times:", df['Wind_Speed_KMH'].isna().sum())

In [ ]:
# Save as CSV
df.to_csv('cleaned_pitstops.csv', index=False)

# For Colab users — this lets you download directly
from google.colab import files
files.download('cleaned_pitstops.csv')

In [ ]:

cols = ['Air_Temp_C', 'Track_Temp_C', 'Humidity_%', 'Wind_Speed_KMH']

# 2. Find rows where ALL 4 are missing
all_missing_mask = df[cols].isna().all(axis=1)
missing_rows = df[all_missing_mask]

print(f"Found {len(missing_rows):,} rows where ALL 4 columns are missing.\n")

# 3. Select only the columns we care about
id_cols = ['Season', 'Round', 'Circuit']
unique_races = missing_rows[id_cols].drop_duplicates()

print(f"Unique (Season, Round, Circuit) combinations: {len(unique_races)}\n")
print(unique_races.to_string(index=False))

In [ ]:
# 1. Define the 4 columns that are missing together
cols = ['Air_Temp_C', 'Track_Temp_C', 'Humidity_%', 'Wind_Speed_KMH']

# 2. Count how many such rows exist
rows_to_remove = df[cols].isna().all(axis=1).sum()
print(f"Rows where ALL 4 columns are missing: {rows_to_remove:,}")

# 3. Drop those rows
df_clean = df[~df[cols].isna().all(axis=1)].copy()

print(f"Before: {len(df):,} rows")
print(f" After: {len(df_clean):,} rows  → removed {rows_to_remove:,} rows")

# 4. Optional: Reassign back to df
df = df_clean

# 5. Final check: no row should have all 4 missing
still_missing = df[cols].isna().all(axis=1).sum()
print(f"Rows with all 4 missing after cleanup: {still_missing}  (should be 0)")

In [ ]:
df = df[~df[['Air_Temp_C', 'Track_Temp_C', 'Humidity_%', 'Wind_Speed_KMH']].isna().all(axis=1)].copy()
print(f"Cleaned! Now {len(df):,} rows")

In [ ]:
print("Remaining missing times:", df['Humidity_%'].isna().sum())

In [ ]:
df.to_csv('cleaned_pitstops.csv', index=False)

# For Colab users — this lets you download directly
from google.colab import files
files.download('cleaned_pitstops.csv')

In [ ]:
# Deep learning regression training script for predicting Driver Aggression Score
# Run in a Python env with pandas, sklearn, matplotlib and tensorflow installed (or it will fallback to sklearn's MLP).

import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.pipeline import Pipeline
import joblib

# --- CONFIG ---
CSV_PATH = "/content/cleaned_pitstops.csv"
TARGET = "Driver Aggression Score"
TEST_SIZE = 0.15
VAL_SIZE = 0.15
RANDOM_STATE = 42
SAVE_DIR = "/tmp/f1_model"   # change as needed
os.makedirs(SAVE_DIR, exist_ok=True)

# --- LOAD ---
df = pd.read_csv(CSV_PATH)
print("Loaded shape:", df.shape)

# drop rows missing target
df = df[~df[TARGET].isna()].copy()
print("After dropping missing target rows:", df.shape)

# Choose feature columns (exclude identifiers and target)
# Customize drop list if needed
drop_cols = ['Driver', 'Date', 'Time_of_race', 'Pit_Time', TARGET]
features = [c for c in df.columns if c not in drop_cols]
print("Using features:", features)

# --- Detect numeric vs categorical (simple heuristic) ---
numeric_cols = []
cat_cols = []
for c in features:
    if pd.api.types.is_numeric_dtype(df[c]):
        numeric_cols.append(c)
    else:
        # if column is convertible to numeric for >90% rows, treat numeric
        coerced = pd.to_numeric(df[c], errors='coerce')
        if coerced.notna().mean() > 0.9:
            df[c] = coerced
            numeric_cols.append(c)
        else:
            cat_cols.append(c)

print("Numeric:", numeric_cols)
print("Categorical:", cat_cols)

# Fill missing values: numbers -> median, categories -> 'MISSING'
df[numeric_cols] = df[numeric_cols].fillna(df[numeric_cols].median())
df[cat_cols] = df[cat_cols].fillna('MISSING')

# Preprocessing: scale numeric, one-hot encode categorical
preprocessor = ColumnTransformer([
    ("num", StandardScaler(), numeric_cols),
    ("cat", OneHotEncoder(handle_unknown="ignore", sparse_output=False), cat_cols)
], remainder="drop")

X = df[features]
y = df[TARGET].astype(float).values

# Fit preprocessor and transform
X_proc = preprocessor.fit_transform(X)
print("Processed feature matrix shape:", X_proc.shape)

# Train/val/test split
X_train_val, X_test, y_train_val, y_test = train_test_split(X_proc, y, test_size=TEST_SIZE, random_state=RANDOM_STATE)
# keep a validation set from train_val
val_fraction_of_train = VAL_SIZE / (1 - TEST_SIZE)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=val_fraction_of_train, random_state=RANDOM_STATE)

print("Train/Val/Test sizes:", X_train.shape, X_val.shape, X_test.shape)

# --- Build Keras model (if TF available), else fallback to sklearn MLP ---
try:
    import tensorflow as tf
    from tensorflow.keras import layers, models, callbacks, optimizers, regularizers
    tf.random.set_seed(RANDOM_STATE)

    def build_model(input_dim):
        inp = layers.Input(shape=(input_dim,))
        x = layers.Dense(512, activation="relu")(inp)
        x = layers.BatchNormalization()(x)
        x = layers.Dropout(0.4)(x)

        x = layers.Dense(256, activation="relu", kernel_regularizer=regularizers.l2(1e-4))(x)
        x = layers.BatchNormalization()(x)
        x = layers.Dropout(0.3)(x)

        x = layers.Dense(128, activation="relu")(x)
        x = layers.BatchNormalization()(x)
        x = layers.Dropout(0.2)(x)

        x = layers.Dense(64, activation="relu")(x)
        x = layers.BatchNormalization()(x)
        x = layers.Dropout(0.1)(x)

        out = layers.Dense(1, activation="linear")(x)
        model = models.Model(inputs=inp, outputs=out)
        return model

    model = build_model(X_train.shape[1])
    model.compile(optimizer=optimizers.Adam(learning_rate=1e-3), loss="mse", metrics=[tf.keras.metrics.MeanAbsoluteError()])

    model.summary()

    # callbacks
    es = callbacks.EarlyStopping(monitor="val_loss", patience=15, restore_best_weights=True, verbose=1)
    rlrop = callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=7, min_lr=1e-6, verbose=1)
    chkpt = callbacks.ModelCheckpoint(os.path.join(SAVE_DIR, "best_model.h5"), monitor="val_loss", save_best_only=True, verbose=0)

    history = model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=200,
        batch_size=32,
        callbacks=[es, rlrop, chkpt],
        verbose=2
    )

    # evaluate
    y_pred = model.predict(X_test).ravel()
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, y_pred)
    print(f"\nTest MAE: {mae:.4f}, MSE: {mse:.4f}, RMSE: {rmse:.4f}, R2: {r2:.4f}")

    # Save model and preprocessor
    model.save(os.path.join(SAVE_DIR, "keras_model"))
    joblib.dump(preprocessor, os.path.join(SAVE_DIR, "preprocessor.joblib"))
    print(f"Saved Keras model and preprocessor to {SAVE_DIR}")

    # --- Plots ---
    # Loss
    plt.figure()
    plt.plot(history.history["loss"], label="train_loss")
    plt.plot(history.history["val_loss"], label="val_loss")
    plt.xlabel("Epoch"); plt.ylabel("MSE Loss"); plt.legend(); plt.title("Training / Validation Loss")
    plt.show()

    # MAE
    if "mean_absolute_error" in history.history:
        train_mae_key = "mean_absolute_error"
        val_mae_key = "val_mean_absolute_error"
    else:
        # Keras metric name sometimes "mean_absolute_error" or "mean_absolute_error"
        train_mae_key = [k for k in history.history.keys() if "mean_absolute" in k and not k.startswith("val")][0]
        val_mae_key = [k for k in history.history.keys() if k.startswith("val") and "mean_absolute" in k][0]

    plt.figure()
    plt.plot(history.history[train_mae_key], label="train_mae")
    plt.plot(history.history[val_mae_key], label="val_mae")
    plt.xlabel("Epoch"); plt.ylabel("MAE"); plt.legend(); plt.title("Training / Validation MAE")
    plt.show()

    # Pred vs Actual
    plt.figure()
    plt.scatter(y_test, y_pred, alpha=0.5)
    plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()])
    plt.xlabel("Actual Driver Aggression Score"); plt.ylabel("Predicted")
    plt.title("Predicted vs Actual (Test)")
    plt.show()

    # Residuals
    residuals = y_test - y_pred
    plt.figure()
    plt.hist(residuals, bins=40)
    plt.xlabel("Residual (Actual - Pred)"); plt.title("Residuals Distribution")
    plt.show()

except Exception as e:
    # Fallback: sklearn MLP (useful if TF not installed)
    print("TensorFlow Keras not available or failed. Falling back to sklearn MLPRegressor. Error:", e)
    from sklearn.neural_network import MLPRegressor

    # Convert one-hot or categorical differently? We already used preprocessor above if created.
    # For speed: use simpler preprocessing: convert categories to codes then scale
    X_simple = df[features].copy()
    # numeric/categorical detection repeated:
    for c in X_simple.columns:
        if not pd.api.types.is_numeric_dtype(X_simple[c]):
            # factorize category -> codes
            X_simple[c] = X_simple[c].astype('category').cat.codes

    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X_simple)

    X_train, X_temp, y_train, y_temp = train_test_split(X_scaled, y, test_size=(TEST_SIZE + VAL_SIZE), random_state=RANDOM_STATE)
    # split val off train
    test_split = TEST_SIZE / (TEST_SIZE + VAL_SIZE)
    X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=test_split, random_state=RANDOM_STATE)

    mlp = MLPRegressor(hidden_layer_sizes=(256,128), max_iter=300, early_stopping=True, n_iter_no_change=10, random_state=RANDOM_STATE)
    mlp.fit(X_train, y_train)

    y_pred = mlp.predict(X_test)
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, y_pred)
    print(f"Fallback MLP Test MAE: {mae:.4f}, MSE: {mse:.4f}, RMSE: {rmse:.4f}, R2: {r2:.4f}")

    # save fallback objects
    joblib.dump(mlp, os.path.join(SAVE_DIR, "mlp_model.joblib"))
    joblib.dump(scaler, os.path.join(SAVE_DIR, "scaler.joblib"))
    print(f"Saved fallback MLP and scaler to {SAVE_DIR}")

    # simple plots
    plt.figure()
    plt.scatter(y_test, y_pred, alpha=0.5)
    plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()])
    plt.title("Predicted vs Actual (Test) - Fallback MLP"); plt.xlabel("Actual"); plt.ylabel("Predicted")
    plt.show()

    plt.figure()
    plt.hist(y_test - y_pred, bins=40)
    plt.title("Residuals - Fallback MLP"); plt.xlabel("Residual")
    plt.show()

# --- Useful metrics outputs ---
print("\nFinal numeric metrics: ")
print("MAE:", mean_absolute_error(y_test, y_pred))
print("MSE:", mean_squared_error(y_test, y_pred))
print("RMSE:", np.sqrt(mean_squared_error(y_test, y_pred)))
print("R2:", r2_score(y_test, y_pred))

# Optional: discretized accuracy (if you want to measure class-like accuracy)
try:
    bins = np.linspace(y.min(), y.max(), 6)  # 5 bins
    y_test_bins = np.digitize(y_test, bins)
    y_pred_bins = np.digitize(y_pred, bins)
    from sklearn.metrics import accuracy_score
    print("Discretized accuracy (5 bins):", accuracy_score(y_test_bins, y_pred_bins))
except Exception:
    pass

print("Script finished. Models and preprocessors saved to", SAVE_DIR)

In [ ]:
import joblib

# Save model to file
joblib.dump(model, "driver_aggression_model.pkl")
print("✅ Model saved successfully as driver_aggression_model.pkl")

In [ ]:
import joblib

# Save model to file
joblib.dump(model, "driver_aggression_model.pkl")
print("✅ Model saved successfully as driver_aggression_model.pkl")

In [ ]:
# ===============================================================
# TABTRANSFORMER MODEL FOR DRIVER AGGRESSION SCORE PREDICTION
# ===============================================================

import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers, callbacks
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import pickle

# ---------------------------------------------------------------
# 1️⃣ LOAD DATASET
# ---------------------------------------------------------------
df = pd.read_csv('/content/cleaned_pitstops.csv')

# ---------------------------------------------------------------
# 2️⃣ SPLIT NUMERIC & CATEGORICAL FEATURES
# ---------------------------------------------------------------
target = 'Driver Aggression Score'
numeric_cols = df.select_dtypes(include=[np.number]).columns.drop(target)
cat_cols = df.select_dtypes(exclude=[np.number]).columns

# ---------------------------------------------------------------
# 3️⃣ ENCODE CATEGORICAL FEATURES
# ---------------------------------------------------------------
cat_encoders = {}
for col in cat_cols:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col].astype(str))
    cat_encoders[col] = le

# ---------------------------------------------------------------
# 4️⃣ SCALE NUMERIC FEATURES
# ---------------------------------------------------------------
scaler = StandardScaler()
df[numeric_cols] = scaler.fit_transform(df[numeric_cols])

# ---------------------------------------------------------------
# 5️⃣ TRAIN-TEST SPLIT
# ---------------------------------------------------------------
X = df.drop(columns=[target])
y = df[target]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Prepare inputs for TabTransformer
X_train_num = X_train[numeric_cols].values
X_test_num = X_test[numeric_cols].values
X_train_cat = [X_train[col].values for col in cat_cols]
X_test_cat = [X_test[col].values for col in cat_cols]

# ---------------------------------------------------------------
# 6️⃣ DEFINE TABTRANSFORMER MODEL
# ---------------------------------------------------------------
def build_tabtransformer(num_numeric, cat_cardinalities, embed_dim=32, num_heads=4, num_layers=3):
    inputs_num = layers.Input(shape=(num_numeric,), name="numeric")

    # Categorical inputs + embeddings
    cat_inputs = [layers.Input(shape=(1,), name=f"cat_{i}") for i in range(len(cat_cardinalities))]
    cat_embeddings = [
        layers.Embedding(input_dim=card, output_dim=embed_dim)(inp)
        for card, inp in zip(cat_cardinalities, cat_inputs)
    ]

    # Concatenate embeddings -> [batch, num_cats, embed_dim]
    cat_features = layers.Concatenate(axis=1)(cat_embeddings)
    if len(cat_features.shape) == 4:
        cat_features = layers.Lambda(lambda x: tf.squeeze(x, axis=2))(cat_features)

    # Project numeric features into same embedding space
    num_proj = layers.Dense(embed_dim)(inputs_num)
    num_proj = layers.Lambda(lambda x: tf.expand_dims(x, axis=1))(num_proj)  # Keras-safe expand

    # Combine numeric + categorical tokens
    tokens = layers.Concatenate(axis=1)([num_proj, cat_features])

    # Transformer encoder layers
    x = tokens
    for _ in range(num_layers):
        attn_out = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)(x, x)
        x = layers.Add()([x, attn_out])
        x = layers.LayerNormalization()(x)
        ff = layers.Dense(embed_dim * 4, activation='relu')(x)
        ff = layers.Dense(embed_dim)(ff)
        x = layers.Add()([x, ff])
        x = layers.LayerNormalization()(x)

    # Pooling + Regression head
    x = layers.GlobalAveragePooling1D()(x)
    x = layers.Dense(128, activation='relu')(x)
    x = layers.Dropout(0.3)(x)
    output = layers.Dense(1, activation='linear', name='Driver_Aggression_Score')(x)

    model = models.Model(inputs=[inputs_num] + cat_inputs, outputs=output)
    return model

# ---------------------------------------------------------------
# 7️⃣ BUILD AND COMPILE MODEL
# ---------------------------------------------------------------
cat_cardinalities = [len(cat_encoders[col].classes_) for col in cat_cols]
model = build_tabtransformer(len(numeric_cols), cat_cardinalities)

model.compile(
    optimizer=optimizers.Adam(1e-3),
    loss='mse',
    metrics=[tf.keras.metrics.MeanAbsoluteError()]
)

model.summary()

# ---------------------------------------------------------------
# 8️⃣ TRAIN MODEL
# ---------------------------------------------------------------
es = callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

history = model.fit(
    [X_train_num] + X_train_cat, y_train,
    validation_data=([X_test_num] + X_test_cat, y_test),
    epochs=100,
    batch_size=32,
    callbacks=[es],
    verbose=1
)

# ---------------------------------------------------------------
# 9️⃣ EVALUATE MODEL
# ---------------------------------------------------------------
results = model.evaluate([X_test_num] + X_test_cat, y_test, verbose=0)
print(f"\n✅ Test Loss (MSE): {results[0]:.4f}")
print(f"✅ Test MAE: {results[1]:.4f}")

# ---------------------------------------------------------------
# 🔟 VISUALIZE TRAINING PERFORMANCE
# ---------------------------------------------------------------
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.xlabel('Epochs')
plt.ylabel('MSE Loss')
plt.legend()
plt.title('Model Loss over Epochs')

plt.subplot(1, 2, 2)
plt.plot(history.history['mean_absolute_error'], label='Train MAE')
plt.plot(history.history['val_mean_absolute_error'], label='Val MAE')
plt.xlabel('Epochs')
plt.ylabel('Mean Absolute Error')
plt.legend()
plt.title('MAE over Epochs')

plt.show()

# ---------------------------------------------------------------
# 1️⃣1️⃣ SAVE MODEL + ENCODERS
# ---------------------------------------------------------------
model.save('driver_aggression_tabtransformer.h5')
print("\n✅ Model saved as 'driver_aggression_tabtransformer.h5'")

with open('encoders_scaler.pkl', 'wb') as f:
    pickle.dump({'cat_encoders': cat_encoders, 'scaler': scaler}, f)

print("✅ Encoders & Scaler saved as 'encoders_scaler.pkl'")

In [ ]:
import pandas as pd
pd.DataFrame(history.history).to_csv("training_metrics.csv", index=False)

In [ ]:
y_pred = model.predict([X_test_num] + X_test_cat).ravel()
plt.scatter(y_test, y_pred, alpha=0.5)
plt.xlabel("Actual Aggression")
plt.ylabel("Predicted Aggression")
plt.title("Predicted vs Actual Aggression Score")
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--')
plt.show()

In [ ]:
from sklearn.metrics import r2_score
print("R² Score:", r2_score(y_test, y_pred))

In [ ]:
model.save("driver_aggression_tabtransformer_v1.keras")

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models

def build_tabtransformer_safe(num_numeric, cat_cardinalities, embed_dim=32, num_heads=4, num_layers=3):
    inputs_num = layers.Input(shape=(num_numeric,), name="numeric")

    # categorical inputs + embeddings
    cat_inputs = [layers.Input(shape=(1,), name=f"cat_{i}") for i in range(len(cat_cardinalities))]
    cat_embeddings = [
        layers.Embedding(input_dim=card, output_dim=embed_dim)(inp)
        for card, inp in zip(cat_cardinalities, cat_inputs)
    ]

    # concatenate embeddings
    cat_features = layers.Concatenate(axis=1)(cat_embeddings)
    cat_features = layers.Lambda(
        lambda x: tf.squeeze(x, axis=2),
        output_shape=lambda s: (s[0], s[1], s[3])
    )(cat_features) if len(cat_features.shape) == 4 else cat_features

    # numeric projection
    num_proj = layers.Dense(embed_dim)(inputs_num)
    num_proj = layers.Lambda(
        lambda x: tf.expand_dims(x, axis=1),
        output_shape=lambda s: (s[0], 1, s[1])
    )(num_proj)

    # combine tokens
    tokens = layers.Concatenate(axis=1)([num_proj, cat_features])

    # transformer encoder
    x = tokens
    for _ in range(num_layers):
        attn_out = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)(x, x)
        x = layers.Add()([x, attn_out])
        x = layers.LayerNormalization()(x)
        ff = layers.Dense(embed_dim * 4, activation='relu')(x)
        ff = layers.Dense(embed_dim)(ff)
        x = layers.Add()([x, ff])
        x = layers.LayerNormalization()(x)

    # pooling + regression
    x = layers.GlobalAveragePooling1D()(x)
    x = layers.Dense(128, activation='relu')(x)
    x = layers.Dropout(0.3)(x)
    output = layers.Dense(1, activation='linear', name='Driver_Aggression_Score')(x)

    model = models.Model(inputs=[inputs_num] + cat_inputs, outputs=output)
    return model

In [ ]:
import joblib

SAVE_DIR = "/tmp/f1_model"

# Save model in Keras 3 format
model.save(f"{SAVE_DIR}/driver_aggression_model.keras")

# Save the preprocessor
joblib.dump(preprocessor, f"{SAVE_DIR}/preprocessor.joblib")
print("✅ Model and preprocessor saved successfully.")

In [ ]:
def build_tabtransformer(num_numeric, cat_cardinalities, embed_dim=32, num_heads=4, num_layers=3):
    inputs_num = layers.Input(shape=(num_numeric,), name="numeric")

    # Categorical inputs + embeddings
    cat_inputs = [layers.Input(shape=(1,), name=f"cat_{i}") for i in range(len(cat_cardinalities))]
    cat_embeddings = [
        layers.Embedding(input_dim=card, output_dim=embed_dim)(inp)
        for card, inp in zip(cat_cardinalities, cat_inputs)
    ]

    # Concatenate embeddings -> [batch, num_cats, embed_dim]
    cat_features = layers.Concatenate(axis=1)(cat_embeddings)
    if len(cat_features.shape) == 4:
        cat_features = layers.Lambda(
            lambda x: tf.squeeze(x, axis=2),
            output_shape=lambda input_shape: (input_shape[0], input_shape[1], input_shape[3])
        )(cat_features)

    # Project numeric features into same embedding space
    num_proj = layers.Dense(embed_dim)(inputs_num)
    num_proj = layers.Lambda(
        lambda x: tf.expand_dims(x, axis=1),
        output_shape=lambda input_shape: (input_shape[0], 1, input_shape[1])
    )(num_proj)

    # Combine numeric + categorical tokens
    tokens = layers.Concatenate(axis=1)([num_proj, cat_features])

    # Transformer encoder layers
    x = tokens
    for _ in range(num_layers):
        attn_out = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)(x, x)
        x = layers.Add()([x, attn_out])
        x = layers.LayerNormalization()(x)
        ff = layers.Dense(embed_dim * 4, activation='relu')(x)
        ff = layers.Dense(embed_dim)(ff)
        x = layers.Add()([x, ff])
        x = layers.LayerNormalization()(x)

    # Pooling + Regression head
    x = layers.GlobalAveragePooling1D()(x)
    x = layers.Dense(128, activation='relu')(x)
    x = layers.Dropout(0.3)(x)
    output = layers.Dense(1, activation='linear', name='Driver_Aggression_Score')(x)

    model = models.Model(inputs=[inputs_num] + cat_inputs, outputs=output)
    return model

In [ ]:
model.save(f"{SAVE_DIR}/driver_aggression_model.keras")

In [ ]:
# ===============================================================
# 🔧 FIXED REBUILD CODE TO LOAD OLD WEIGHTS SAFELY
# ===============================================================

import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
import pickle

# 1️⃣ Reload your dataset and metadata
df = pd.read_csv('/content/cleaned_pitstops.csv')

target = 'Driver Aggression Score'
numeric_cols = df.select_dtypes(include=[np.number]).columns.drop(target)
cat_cols = df.select_dtypes(exclude=[np.number]).columns

# Load encoders/scaler to get categorical sizes
with open('encoders_scaler.pkl', 'rb') as f:
    obj = pickle.load(f)
cat_encoders = obj['cat_encoders']
scaler = obj['scaler']

cat_cardinalities = [len(cat_encoders[col].classes_) for col in cat_cols]
num_numeric = len(numeric_cols)

# 2️⃣ Rebuild architecture (with output_shape fixes)
def build_tabtransformer(num_numeric, cat_cardinalities, embed_dim=32, num_heads=4, num_layers=3):
    inputs_num = layers.Input(shape=(num_numeric,), name="numeric")
    cat_inputs = [layers.Input(shape=(1,), name=f"cat_{i}") for i in range(len(cat_cardinalities))]

    cat_embeddings = [
        layers.Embedding(input_dim=card, output_dim=embed_dim)(inp)
        for card, inp in zip(cat_cardinalities, cat_inputs)
    ]

    cat_features = layers.Concatenate(axis=1)(cat_embeddings)
    if len(cat_features.shape) == 4:
        cat_features = layers.Lambda(
            lambda x: tf.squeeze(x, axis=2),
            output_shape=lambda s: (s[0], s[1], s[3])
        )(cat_features)

    num_proj = layers.Dense(embed_dim)(inputs_num)
    num_proj = layers.Lambda(
        lambda x: tf.expand_dims(x, axis=1),
        output_shape=lambda s: (s[0], 1, s[1])
    )(num_proj)

    tokens = layers.Concatenate(axis=1)([num_proj, cat_features])

    x = tokens
    for _ in range(num_layers):
        attn_out = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)(x, x)
        x = layers.Add()([x, attn_out])
        x = layers.LayerNormalization()(x)
        ff = layers.Dense(embed_dim * 4, activation='relu')(x)
        ff = layers.Dense(embed_dim)(ff)
        x = layers.Add()([x, ff])
        x = layers.LayerNormalization()(x)

    x = layers.GlobalAveragePooling1D()(x)
    x = layers.Dense(128, activation='relu')(x)
    x = layers.Dropout(0.3)(x)
    output = layers.Dense(1, activation='linear')(x)

    model = models.Model(inputs=[inputs_num] + cat_inputs, outputs=output)
    return model

# 3️⃣ Rebuild and load your weights
model = build_tabtransformer(num_numeric, cat_cardinalities)
model.load_weights("/content/driver_aggression_tabtransformer.h5")

# 4️⃣ Save in new format (no shape inference issues)
SAVE_DIR = "/tmp/f1_model"
model.save(f"{SAVE_DIR}/driver_aggression_model.keras")

print("✅ Fixed model successfully rebuilt and saved to /tmp/f1_model/driver_aggression_model.keras")

In [ ]:
model = tf.keras.models.load_model(f"{SAVE_DIR}/driver_aggression_model.keras")
preprocessor = joblib.load(f"{SAVE_DIR}/preprocessor.joblib")

In [ ]:
import pandas as pd
import numpy as np
import joblib

# Load dataset again
df = pd.read_csv('/content/cleaned_pitstops.csv')
target = 'Driver Aggression Score'

# Separate columns
numeric_cols = df.select_dtypes(include=[np.number]).columns.drop(target)
cat_cols = df.select_dtypes(exclude=[np.number]).columns

# Load encoders and scaler
with open('encoders_scaler.pkl', 'rb') as f:
    obj = pickle.load(f)
cat_encoders = obj['cat_encoders']
scaler = obj['scaler']

# Apply encoders + scaler again (same preprocessing)
for col in cat_cols:
    df[col] = cat_encoders[col].transform(df[col].astype(str))
df[numeric_cols] = scaler.transform(df[numeric_cols])

# Split data
from sklearn.model_selection import train_test_split
X = df.drop(columns=[target])
y = df[target]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
import tensorflow as tf
from tensorflow import keras

SAVE_DIR = "/tmp/f1_model"

# ✅ Reload the model safely, giving Keras a reference to TensorFlow
custom_objects = {"tf": tf}
model = keras.models.load_model(
    f"{SAVE_DIR}/driver_aggression_model.keras",
    custom_objects=custom_objects
)

print("✅ Model reloaded successfully with TensorFlow bound to Lambda layers.")

In [ ]:
import tensorflow as tf
from tensorflow import keras

SAVE_DIR = "/tmp/f1_model"

# Explicitly give TensorFlow reference to Keras during deserialization
model = keras.models.load_model(
    f"{SAVE_DIR}/driver_aggression_model.keras",
    custom_objects={"tf": tf}
)

print("✅ Model loaded successfully with TensorFlow bound.")

In [ ]:
# Preprocessor should already be loaded
# Example: preprocessor = joblib.load(f"{SAVE_DIR}/preprocessor.joblib")

def model_predict(X):
    # Make sure X is transformed before prediction
    X_proc = preprocessor.transform(X)
    preds = model.predict(X_proc)
    return preds

In [ ]:
import numpy as np
import pandas as pd
import shap
import pickle

# -------------------------------
# Load saved encoders + scaler
# -------------------------------
with open("encoders_scaler.pkl", "rb") as f:
    enc_dict = pickle.load(f)

cat_encoders = enc_dict['cat_encoders']
scaler = enc_dict['scaler']

# -------------------------------
# Prepare sample data
# -------------------------------
X_sample = X_test.sample(200, random_state=42).copy()

# Encode categorical columns
for col, le in cat_encoders.items():
    X_sample[col] = X_sample[col].astype(str)
    # Replace unseen labels with a known one (to avoid errors)
    X_sample[col] = X_sample[col].apply(lambda x: x if x in le.classes_ else le.classes_[0])
    X_sample[col] = le.transform(X_sample[col])

# Scale numeric columns
X_sample[numeric_cols] = scaler.transform(X_sample[numeric_cols])

# Convert to numpy for model
X_num = X_sample[numeric_cols].values
X_cat = [X_sample[col].values for col in cat_encoders.keys()]
X_combined = [X_num] + X_cat

print("✅ Manual preprocessing completed:", X_num.shape, "numeric +", len(X_cat), "categorical")

# -------------------------------
# SHAP explainability
# -------------------------------
print("⚙️ Running SHAP explainability (may take 2–3 min)...")

# Use a small background for speed
X_bg_num = X_num[:50]
X_bg_cat = [x[:50] for x in X_cat]
X_bg = [X_bg_num] + X_bg_cat

# Wrap prediction
def model_predict(data):
    num_features = data[:, :X_num.shape[1]]
    cat_features = [data[:, X_num.shape[1] + i] for i in range(len(X_cat))]
    return model.predict([num_features] + cat_features)

# Combine numeric + categorical for SHAP
X_shap = np.hstack([X_num] + [c.reshape(-1, 1) for c in X_cat])

# Run SHAP
explainer = shap.KernelExplainer(model_predict, X_shap[:50])
shap_values = explainer.shap_values(X_shap[:100])

# -------------------------------
# Visualize
# -------------------------------
shap.summary_plot(shap_values, X_shap)
shap.plots.bar(shap_values)

In [ ]:
print("shap_values shape:", np.array(shap_values).shape)
print("X_shap shape:", X_shap_sample.shape)

In [ ]:
shap_values = np.squeeze(shap_values, axis=-1)

# Now visualize
shap.summary_plot(shap_values, X_shap_sample)
shap.plots.bar(shap_values)

In [ ]:
# ✅ Save model
model.save("final_model.h5")
print("✅ Model saved as final_model.h5")

# ✅ Load model later
from tensorflow.keras.models import load_model
model = load_model("final_model.h5")
print("✅ Model loaded successfully")

In [ ]:
# --------------------------------------------
# ✅ Recover feature names from your preprocessor
# --------------------------------------------
import numpy as np

# Numerical and categorical feature lists
num_features = preprocessor.transformers_[0][2]
cat_features = preprocessor.transformers_[1][2]

# Get encoded categorical feature names
cat_encoder = preprocessor.named_transformers_['cat'].named_steps['onehot']
cat_encoded_names = list(cat_encoder.get_feature_names_out(cat_features))

# Combine them all
all_feature_names = list(num_features) + cat_encoded_names

print("🧩 Total processed features:", len(all_feature_names))
print("🔹 First 15 feature names:\n", all_feature_names[:15])